In [1]:
import os

HF_TOKEN = os.getenv("HF_TOKEN")
print(HF_TOKEN)

None


In [8]:
checkpoint = "gpt2"
checkpoint_assist = "distilgpt2"
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Select device (GPU or CPU)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_auth_token=HF_TOKEN)
main_model = AutoModelForCausalLM.from_pretrained(checkpoint, use_auth_token=HF_TOKEN).to(device)
assistant_model = AutoModelForCausalLM.from_pretrained(checkpoint_assist, use_auth_token=HF_TOKEN).to(device)


c:\Users\Franek\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

c:\Users\Franek\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Franek\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

: 

In [7]:
import time
def vanilla_generation(model, tokenizer, prompt, max_tokens=8):
    start = time.time()
    input = tokenizer(prompt, return_tensors="pt")
    input = input.to(device)
    out = model.generate(**input, max_new_tokens=max_tokens)
    end = time.time()
    print(tokenizer.decode(out[0], skip_special_tokens=True))
    return end - start


In [ ]:
def speculative_decoding(tokenizer, model, assistant_model, prompt, max_len, speculative_len=5):
    # Generating tokens we will speculate on:
    cur_len = 0
    input = tokenizer(prompt, return_tensors = "pt")
    input = input.to(device)
    while cur_len < max_len:
        for _ in range(speculative_len):
            with torch.no_grad():
                out = assistant_model(input)
                next_token = out.logits[:, -1, :].argmax(dim=-1)
                input = torch.cat((input, next_token[:, None]), dim=-1)
        
        with torch.no_grad():
            #verifying using main model:
            out_logits = model(input)
            out_logits = out_logits[:, -speculative_len, :]
            out = torch.argmax(out_logits, dim=-1)
            
